In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Save zarr stores for some variables
Chug certain variables into a zarr store in `/work` for later use.

In [2]:
import xarray as xr
import gfdl_utils as gu
from dask.diagnostics import ProgressBar

In [3]:
config_id = 'ESM4_historical_D1'
outdir = '/work/gam/projects/oxygen/data/'
filename = 'ds.'+config_id+'.zarr'

In [21]:
dmget = True
pathDict = {'pp':('/archive/oar.gfdl.cmip6/ESM4/DECK/'+config_id+'/'+
                        'gfdl.ncrc4-intel16-prod-openmp/pp'),
                'out':'ts',
                'time':'*'}

In [22]:
want_1x1deg = False
want_timefrequency = 'monthly'
variables = ['o2','o2sat','fgo2','thetao','so','volcello','wc_vert_int_jo2resp']
ppnameuse = {}
ds = xr.Dataset()
for variable in variables:
    ppnames = gu.core.find_variable(pathDict['pp'],variable)
    for ppname in ppnames:
        timefrequency = gu.core.get_timefrequency(pathDict['pp'],ppname)
        deg = gu.core.query_is1x1deg(ppname)
        if (timefrequency==want_timefrequency) & (deg==want_1x1deg):
            ppnameuse[variable]=ppname
    pathDict['local']=gu.core.get_local(pathDict['pp'],ppnameuse[variable],pathDict['out'])
    if dmget:
        path = gu.core.get_pathspp(**pathDict,ppname=ppnameuse[variable],add=variable)
        gu.core.issue_dmget(path)
    else:
        ds[variable] = gu.core.open_frompp(**pathDict,ppname=ppname,add=variable)[variable]

sgi_dmf-9 DMF daemon: warning 
  fhandle 010000000000001893316e1b3b41b9550000002200007cce7993da7300000000, file /arch0/cm6/ESM4/DECK/ESM4_historical_D1/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_cobalt_omip_tracers_month_z/ts/monthly/5yr/ocean_cobalt_omip_tracers_month_z.201001-201412.o2.nc, The DMF daemon is unable to recall the requested file.
sgi_dmf-9 DMF daemon: warning 
  fhandle 010000000000001893316e1b3b41b9550000002200007ccf7993da7300000000, file /arch0/cm6/ESM4/DECK/ESM4_historical_D1/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_cobalt_omip_tracers_month_z/ts/monthly/5yr/ocean_cobalt_omip_tracers_month_z.201001-201412.o2sat.nc, The DMF daemon is unable to recall the requested file.


In [24]:
ppnameuse

{'o2': 'ocean_cobalt_omip_tracers_month_z',
 'o2sat': 'ocean_cobalt_omip_tracers_month_z',
 'fgo2': 'ocean_cobalt_omip_2d',
 'thetao': 'ocean_monthly_z',
 'so': 'ocean_monthly_z',
 'volcello': 'ocean_inert_z',
 'wc_vert_int_jo2resp': 'ocean_cobalt_fluxes_int'}

sgi_dmf-9 DMF daemon: warning 
  fhandle 010000000000001893316e1b3b41b955000000ff000295150fce6d7700000000, file /arch0/cm6/ESM4/DECK/ESM4_historical_D1/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/ocean_monthly_z.193001-193412.thetao.nc, The DMF daemon is unable to recall the requested file.
sgi_dmf-9 DMF daemon: warning 
  fhandle 010000000000001893316e1b3b41b955000000ff00029480af18b37100000000, file /arch0/cm6/ESM4/DECK/ESM4_historical_D1/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/ocean_monthly_z.187001-187412.thetao.nc, The DMF daemon is unable to recall the requested file.


In [12]:
gu.core.get_timefrequency(pathDict['pp'],'ocean_cobalt_omip_tracers_year_z')

'annual'

In [14]:
gu.core.query_is1x1deg('ocean_cobalt_omip_tracers_month_z_1x1deg')

True

In [120]:
variable = 'fgo2'
ppname = 'ocean_cobalt_omip_2d'
path = gu.core.get_pathspp(**pathDict,ppname=ppname,add=variable)
# path
ds = xr.Dataset()
ds[variable] = gu.core.open_frompp(**pathDict,ppname=ppname,add=variable)[variable]

In [122]:
with ProgressBar():
    ds.to_zarr(outdir+filename,mode='a')

[########################################] | 100% Completed |  2.8s


In [4]:
dmget = False

ds = xr.Dataset()
if (config_id == 'ESM4_piControl_D') or (config_id == 'ESM4_historical_D1'):

    pathDict = {'pp':('/archive/oar.gfdl.cmip6/ESM4/DECK/'+config_id+'/'+
                        'gfdl.ncrc4-intel16-prod-openmp/pp'),
                'out':'ts',
                'local':'annual/5yr',
                'time':'*'}
    pathDict['ppname']='ocean_cobalt_omip_tracers_year_z_1x1deg'
    variables = ['o2','o2sat']
    for variable in variables:
        pathDict['add'] = variable
        path=gu.core.get_pathspp(**pathDict)
        if dmget:
            gu.core.issue_dmget(path)
        else:
            ds[variable] = gu.core.open_frompp(**pathDict)[variable]
    
    pathDict['ppname']='ocean_annual_z_1x1deg'
    variables = ['thetao','so','volcello']
    for variable in variables:
        pathDict['add'] = variable
        path=gu.core.get_pathspp(**pathDict)
        if dmget:
            gu.core.issue_dmget(path)
        else:
            ds[variable] = gu.core.open_frompp(**pathDict)[variable]
    gridpath=gu.core.get_pathstatic(pathDict['pp'],pathDict['ppname'])
    if dmget:
        gu.core.issue_dmget(gridpath)
    else:
        grid = xr.open_dataset(gridpath)
    
if dmget==False:
    ds = ds.chunk({'time':1}).squeeze()

In [5]:
with ProgressBar():
    ds.to_zarr(outdir+filename,mode='w')

[########################################] | 100% Completed | 20.3s


In [117]:
gu.core.find_variable(pathDict['pp'],'fgo2')

['ocean_cobalt_omip_2d', 'ocean_cobalt_omip_2d_1x1deg']

In [123]:
ds = xr.open_zarr(outdir+filename)

In [124]:
ds

<xarray.Dataset>
Dimensions:   (time: 165, yh: 576, xh: 720, lat: 180, lon: 360, z_l: 35)
Coordinates:
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon       (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * time      (time) object 1850-07-02 12:00:00 ... 2014-07-02 12:00:00
  * xh        (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * yh        (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
  * z_l       (z_l) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
Data variables:
    fgo2      (time, yh, xh) float32 dask.array<chunksize=(5, 576, 720), meta=np.ndarray>
    o2        (time, z_l, lat, lon) float32 dask.array<chunksize=(1, 35, 180, 360), meta=np.ndarray>
    o2sat     (time, z_l, lat, lon) float32 dask.array<chunksize=(1, 35, 180, 360), meta=np.ndarray>
    so        (time, z_l, lat, lon) float32 dask.array<chunksize=(1, 35, 180, 360), meta=np.ndarray>
    thetao    (time, z_l, lat, lon) float32 dask.array<chunksize=(1, 35, 180, 360), meta=np.ndarray>
    volcello  (time, z_l, lat, lon) float32 dask.array<chunksize=(1, 35, 180, 360), meta=np.ndarray>